In [ ]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# Simultaneous Coloring of Approximated Derivatives

In OpenMDAO, partial derivatives for components and semi-total derivatives for groups can be approximated using either finite difference or complex step.  Sometimes the partial or semi-total jacobians in these cases are sparse, and the computing of these jacobians can be made more efficient using simultaneous derivative coloring.  For an explanation of a similar coloring for *total* derivatives, see :ref:`Simultaneous Coloring For Separable Problems<feature_simul_coloring>`.  Finite difference and complex step only work in forward mode, so only a forward mode coloring is possible when using them, but depending on the sparsity pattern of the jacobian, it may still be possible to get significant efficiency gains.


## Dynamic Coloring

Setting up a problem to use dynamic coloring of approximated derivatives requires a call to the `declare_coloring` function.


.. automethod:: openmdao.core.system.System.declare_coloring
    :noindex:


For example, the code below sets up coloring for partial derivatives of outputs of `comp` with respect to inputs of `comp` starting with 'x'. Let's assume here that `MyComp` is an `ExplicitComponent`.  If it were an `ImplicitComponent`, then the wildcard pattern 'x*' would be applied to all inputs *and* outputs (states) of `comp`.

```python
    comp = prob.model.add_subsystem('comp', MyComp())
    comp.declare_coloring('x*', method='cs', num_full_jacs=2, min_improve_pct=10.)
```


Note that in the call to `declare_coloring`, we also set `num_full_jacs` to 2.  This means
that the first 2 times that a partial jacobian is computed for 'comp', it's nonzero values will be computed
without coloring and stored.  Just prior to the 3rd time, the jacobian's sparsity pattern will be computed, which then allows the coloring to be computed and used for the rest of the run. We also set `min_improve_pct` to 10, meaning that if the computed coloring does not reduce the number of nonlinear solves required to compute `comp's` partial jacobian by 10 percent, then `comp` will not use coloring at all.

Semi-total derivative coloring can be performed in a similar way, except that `declare_coloring` would be called on a `Group` instead of a `Component`. `num_full_jacs` can also be passed as an arg to `declare_coloring` on the `Group`.

The purpose of `declare_coloring` is to provide all of the necessary information to allow
OpenMDAO to generate a coloring, either dynamically or manually using `openmdao partial_coloring`.

Coloring files that are generated dynamically will be placed in the directory specified in `problem.options['coloring_dir']` and will be named based on the value of the `per_instance` arg passed to `declare_coloring`.  If `per_instance` is True, the file will be named based on the full pathname of the component or group being colored.  If False, the name will be based on the full module pathname of the class of the given
component or group.

`declare_coloring` should generally be called in the `setup` function of the component or group.

Note that computing a partial jacobian when the jacobian is very large can be quite expensive, even if the jacobian is sparse, because a solution must be computed for every column of the jacobian, so you should set `num_full_jacs` only as high as is necessary to ensure that non-constant computed zeros in the jacobian are unlikely.  OpenMDAO injects random noise into the inputs when solving for the columns of the jacobian, which should make non-constant computed zeros fairly unlikely even for `num_full_jacs=1`.


Here's a modified version of our total coloring example, where we replace one of our components with one that computes a dynamic partial coloring.  A total coloring is also performed, as in the previous example, but this time the total coloring uses sparsity information computed by our component during its dynamic partial coloring.



In [ ]:
class DynamicPartialsComp(om.ExplicitComponent):
    def __init__(self, size):
        super().__init__()
        self.size = size
        self.num_computes = 0

    def setup(self):
        self.add_input('y', np.ones(self.size))
        self.add_input('x', np.ones(self.size))
        self.add_output('g', np.ones(self.size))

        self.declare_partials('*', '*', method='cs')

        # turn on dynamic partial coloring
        self.declare_coloring(wrt='*', method='cs', perturb_size=1e-5, num_full_jacs=1, tol=1e-20,
                              show_summary=True, show_sparsity=True)

    def compute(self, inputs, outputs):
        outputs['g'] = np.arctan(inputs['y'] / inputs['x'])
        self.num_computes += 1
